# Emojify! 

In this notebook i will implement an important application of NLP Sentiment Analysis,Our goal is determine the sentiment of a sentence for example does the sentence repersent happiness or sadness ? to make it more modern we are going to experess the sentence sentiment with an emoijy (Emojifier). 

* The Emojifier app will implement a model which inputs a sentence (such as "Let's go see the baseball game tonight!") and finds the most appropriate emoji to be used with this sentence (⚾️)
So rather than writing:
>"Congratulations on the promotion! Let's get coffee and talk. Love you!"   

The emojifier can automatically turn this into:
>"Congratulations on the promotion! 👍 Let's get coffee and talk. ☕️ Love you! ❤️"

In [23]:
import numpy as np
from emo_utils import *
import emoji
import matplotlib.pyplot as plt

%matplotlib inline

##  Dataset EMOJISET

We have a tiny dataset (X, Y) where:
- X contains 127 sentences (strings).
- Y contains an integer label between 0 and 4 corresponding to an emoji for each sentence.

<img src="images/data_set.png" style="width:700px;height:300px;">
<caption><center> **Figure 1**: EMOJISET - a classification problem with 5 classes. A few examples of sentences are given here. </center></caption>

Let's load the dataset using the code below. We split the dataset between training (127 examples) and testing (56 examples).

In [24]:
X_train, Y_train = read_csv('data/train_emoji.csv')
X_test, Y_test = read_csv('data/tesss.csv')

In [25]:
maxLen = len(max(X_train, key=len).split())

sentences from X_train and corresponding labels from Y_train

In [26]:
for idx in range(10):
    print(X_train[idx], label_to_emoji(Y_train[idx]))

never talk to me again 😞
I am proud of your achievements 😄
It is the worst day in my life 😞
Miss you so much ❤️
food is life 🍴
I love you mum ❤️
Stop saying bullshit 😞
congratulations on your acceptance 😄
The assignment is too long  😞
I want to go play ⚾


## Emojifier Using LSTMs in Keras: 

Let's build an LSTM model that takes word **sequences** as input!

* Emojifier will use pre-trained word embeddings to represent words.
* We will feed word embeddings into an LSTM.
* The LSTM will learn to predict the most appropriate emoji. 
* This model will be able to account for the word ordering. 

Word Emmbeddings is Vectors used to reperesent what the words mean and their correlation with each other for example: if we input a sentence man is to woman king is to ?? , the Emmbedding vector should learn that the answer is queen so the vectors measure some features for each word for example word man it learns features like these is it male -> 1, is it female -> 0 is it food -> 0 and so on 

We use these vectors to learn meanings of words in the sentences inputted to the model

In [27]:
import numpy as np
np.random.seed(0)
from keras.models import Model
from keras.layers import Dense, Input, Dropout, LSTM, Activation
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.initializers import glorot_uniform
np.random.seed(1)

### 2.1 - Overview of the model

Here is the Model we will implement:

<img src="images/emojifier-v2.png" style="width:700px;height:400px;"> <br>
<caption><center> **Figure 2**: Emojifier. A 2-layer LSTM sequence classifier. </center></caption>



##### Network Structure
* The `Embedding()` layer's input is an integer matrix of size **(batch size, max input length)**. 
    * This Layer computes Word Emmbeddings for input sentence
    * This input corresponds to sentences converted into lists of indices (integers). so we will build a function to map sentences to indices

* These Computed Word Emmbeddings will be pushd into LSTM (Long Short Term Memoery) Reccurent neural network to make a prediction about what emoji reperesent the sentence

* Dropout Regularization will be used to reduce Over-fiiting

* The Output will fed again to a Series of LSTMs and then We will apply Regularization again and finally softmax output to make the final prediction 

We will implement the above model in keras using mini-batches to train the model 
   * Keras require that all sequences in the same mini-batch have the **same length**
   * The common solution to handling sequences of **different length** is to use padding.  Specifically:
   * Set a maximum sequence length
   * Pad all sequences to have the same length.

Training Emmbedding Vectors is expensive and requries alot of time so in the next cell i will load pre-trained Emmbedding Vectors and use it to train the model

In [28]:
def read_glove_vecs(glove_file):
    with open(glove_file,'r', encoding="utf8") as f:
        words = set()
        word_to_vec_map = {}
        for line in f:
            line = line.strip().split()
            curr_word = line[0]
            words.add(curr_word)
            word_to_vec_map[curr_word] = np.array(line[1:], dtype=np.float64)
        
        i = 1
        words_to_index = {}
        index_to_words = {}
        for w in sorted(words):
            words_to_index[w] = i
            index_to_words[i] = w
            i = i + 1
    return words_to_index, index_to_words, word_to_vec_map

In [29]:
word_to_index, index_to_word, word_to_vec_map = read_glove_vecs('data\\glove.6B.50d.txt')

In [30]:
# sentences_to_indices

def sentences_to_indices(X, word_to_index, max_len):
    """
    Converts an array of sentences (strings) into an array of indices corresponding to words in the sentences.
    The output shape should be such that it can be given to `Embedding()` (described in Figure 4). 
    
    Arguments:
    X -- array of sentences (strings), of shape (m, 1)
    word_to_index -- a dictionary containing the each word mapped to its index
    max_len -- maximum number of words in a sentence. You can assume every sentence in X is no longer than this. 
    
    Returns:
    X_indices -- array of indices corresponding to words in the sentences from X, of shape (m, max_len)
    """
    
    m = X.shape[0]                                   # number of training examples
    
    ### START CODE HERE ###
    # Initialize X_indices as a numpy matrix of zeros and the correct shape (≈ 1 line)
    X_indices = np.zeros([m,max_len])
    
    for i in range(m):                               # loop over training examples
        
        # Convert the ith training sentence in lower case and split is into words. You should get a list of words.
        sentence_words = [w.lower() for w in X[i].split()] 
        
        # Initialize j to 0
        j = 0
        
        # Loop over the words of sentence_words
        for w in sentence_words:
            # Set the (i,j)th entry of X_indices to the index of the correct word.
            X_indices[i, j] = word_to_index[w]
            # Increment j to j + 1
            j += 1
            
    ### END CODE HERE ###
    
    return X_indices

In [31]:
X1 = np.array(["funny lol", "lets play baseball", "food is ready for you"])
X1_indices = sentences_to_indices(X1,word_to_index, max_len = 5)
print("X1 =", X1)
print("X1_indices =\n", X1_indices)

X1 = ['funny lol' 'lets play baseball' 'food is ready for you']
X1_indices =
 [[155345. 225122.      0.      0.      0.]
 [220930. 286375.  69714.      0.      0.]
 [151204. 192973. 302254. 151349. 394475.]]


#### Build embedding layer

* Let's build the `Embedding()` layer in Keras, using pre-trained word vectors. 
* The embedding layer takes as input a list of word indices.
    * `sentences_to_indices()` creates these word indices.
* The embedding layer will return the word embeddings for a sentence. 


In [32]:
def pretrained_embedding_layer(word_to_vec_map, word_to_index):
    """
    Creates a Keras Embedding() layer and loads in pre-trained GloVe 50-dimensional vectors.
    
    Arguments:
    word_to_vec_map -- dictionary mapping words to their GloVe vector representation.
    word_to_index -- dictionary mapping from words to their indices in the vocabulary (400,001 words)

    Returns:
    embedding_layer -- pretrained layer Keras instance
    """
    
    vocab_len = len(word_to_index) + 1                  # adding 1 to fit Keras embedding (requirement)
    emb_dim = word_to_vec_map["cucumber"].shape[0]      # define dimensionality of your GloVe word vectors (= 50)
    
    
    emb_matrix = np.zeros([vocab_len,emb_dim])
    
    # Set each row "idx" of the embedding matrix to be 
    # the word vector representation of the idx'th word of the vocabulary
    for word, idx in word_to_index.items():
        emb_matrix[idx, :] = word_to_vec_map[word]

    # Define Keras embedding layer with the correct input and output sizes
    # non-trainable.
    embedding_layer = Embedding(
    vocab_len,
    emb_dim,
    trainable = False)

    # Build the embedding layer. 
    embedding_layer.build((None,)) 
    
    # Set the weights of the embedding layer to the embedding matrix.
    embedding_layer.set_weights([emb_matrix])
    
    return embedding_layer

In [33]:
embedding_layer = pretrained_embedding_layer(word_to_vec_map, word_to_index)
print("weights[0][1][3] =", embedding_layer.get_weights()[0][1][3])

weights[0][1][3] = -0.3403


## Building the Emojifier

Lets now build the Emojifier model we previouly showed in figure 2

In [34]:
def Emojify(input_shape, word_to_vec_map, word_to_index):
    """
    Function creating the Emojify model's graph.
    
    Arguments:
    input_shape -- shape of the input, usually (max_len,)
    word_to_vec_map -- dictionary mapping every word in a vocabulary into its 50-dimensional vector representation
    word_to_index -- dictionary mapping from words to their indices in the vocabulary (400,001 words)

    Returns:
    model -- a model instance in Keras
    """
    
    # Define sentence_indices as the input of the graph.
    sentence_indices = Input(shape=input_shape, dtype='int32')
    
    # Create the embedding layer pretrained with GloVe Vectors 
    embedding_layer = pretrained_embedding_layer(word_to_vec_map, word_to_index)
    
    # Propagate sentence_indices through your embedding layer
    embeddings = embedding_layer(sentence_indices)  
    
    # Propagate the embeddings through an LSTM layer with 128-dimensional hidden state
    # The returned output is a batch of sequences.
    X = LSTM(units = 128, return_sequences= True)(embeddings)
    # Add dropout 
    X = Dropout(rate = 0.5)(X)
    # Propagate X trough another LSTM layer with 128-dimensional hidden state
    # The returned output is a single hidden state
    X = LSTM(units = 128, return_sequences= False)(X)
    # Add dropout 
    X = Dropout(rate = 0.5)(X)
    # Propagate X through a Dense layer with 5 units
    X = Dense(units = 5,activation='softmax')(X)
    # Add a softmax activation
    X = Activation('softmax')(X)
    
    # Create Model instance which converts sentence_indices into X.
    model = Model(inputs= sentence_indices, outputs= X)
    
    
    return model

In [35]:
model = Emojify((maxLen,), word_to_vec_map, word_to_index)
model.summary()

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 10)                0         
_________________________________________________________________
embedding_5 (Embedding)      (None, 10, 50)            20000050  
_________________________________________________________________
lstm_5 (LSTM)                (None, 10, 128)           91648     
_________________________________________________________________
dropout_5 (Dropout)          (None, 10, 128)           0         
_________________________________________________________________
lstm_6 (LSTM)                (None, 128)               131584    
_________________________________________________________________
dropout_6 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 5)                 645 

We will Compile our model using `categorical_crossentropy` loss, `adam` optimizer and `['accuracy']` metrics

In [36]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

It's time to train our model. Emojifier `model` takes as input an array of shape (`m`, `max_len`) and outputs probability vectors of shape (`m`, `number of classes`). We thus have to convert X_train (array of sentences as strings) to X_train_indices (array of sentences as list of word indices), and Y_train (labels as indices) to Y_train_oh (labels as one-hot vectors).

In [37]:
X_train_indices = sentences_to_indices(X_train, word_to_index, maxLen)
Y_train_oh = convert_to_one_hot(Y_train, C = 5)

Now let's Fit the Keras model on `X_train_indices` and `Y_train_oh`. We will use `epochs = 50` and `batch_size = 32`.

In [38]:
model.fit(X_train_indices, Y_train_oh, epochs = 50, batch_size = 32, shuffle=True)

Epoch 1/50
132/132 [==============================] - 2s 17ms/step - loss: 1.6085 - accuracy: 0.1818
Epoch 2/50
132/132 [==============================] - 0s 1ms/step - loss: 1.5825 - accuracy: 0.3333
Epoch 3/50
132/132 [==============================] - 0s 1ms/step - loss: 1.5681 - accuracy: 0.2576
Epoch 4/50
132/132 [==============================] - 0s 1ms/step - loss: 1.5504 - accuracy: 0.3561
Epoch 5/50
132/132 [==============================] - 0s 1ms/step - loss: 1.5343 - accuracy: 0.3258
Epoch 6/50
132/132 [==============================] - 0s 1ms/step - loss: 1.5064 - accuracy: 0.4091
Epoch 7/50
132/132 [==============================] - 0s 1ms/step - loss: 1.5003 - accuracy: 0.3712
Epoch 8/50
132/132 [==============================] - 0s 1ms/step - loss: 1.4391 - accuracy: 0.4621
Epoch 9/50
132/132 [==============================] - 0s 1ms/step - loss: 1.4020 - accuracy: 0.5227
Epoch 10/50
132/132 [==============================] - 0s 1ms/step - loss: 1.3475 - accuracy: 0.643

The model Accuracy on the training set is 93% now let's evaluate the model on the test set

In [39]:
X_test_indices = sentences_to_indices(X_test, word_to_index, max_len = maxLen)
Y_test_oh = convert_to_one_hot(Y_test, C = 5)
loss, acc = model.evaluate(X_test_indices, Y_test_oh)
print()
print("Test accuracy = ", acc)

56/56 [==============================] - 0s 8ms/step

Test accuracy =  0.8214285969734192


Not bad on the test set but the model is overfitting so we can tune it further to reduce this overfitting and optimize the test set accuracy

let's see some predictions for the model in the next cells

In [46]:
# Change the sentence below to see your prediction. Make sure all the words are in the Glove embeddings.  
x_test = np.array(['not feeling happy'])
X_test_indices = sentences_to_indices(x_test, word_to_index, maxLen)
print(x_test[0] +' '+  label_to_emoji(np.argmax(model.predict(X_test_indices))))

not feeling happy 😞


In [47]:
x_test = np.array(['feeling good'])
X_test_indices = sentences_to_indices(x_test, word_to_index, maxLen)
print(x_test[0] +' '+  label_to_emoji(np.argmax(model.predict(X_test_indices))))

feeling good 😄


In [48]:
x_test = np.array(['i love you'])
X_test_indices = sentences_to_indices(x_test, word_to_index, maxLen)
print(x_test[0] +' '+  label_to_emoji(np.argmax(model.predict(X_test_indices))))

i love you ❤️


In [55]:
x_test = np.array(['lets eat'])
X_test_indices = sentences_to_indices(x_test, word_to_index, maxLen)
print(x_test[0] +' '+  label_to_emoji(np.argmax(model.predict(X_test_indices))))

lets eat 🍴


We can further optimize and make our app more advanced if we train on larger dataset that contains much more emoji's 